Bibliothèques utiles :

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cfd_common_article import *
from math import *
import time
from scipy.linalg import solve_banded

In [2]:
%matplotlib qt

In [3]:
plt.ion()

Choix du domaine et des paramètres de la discrétisation du problème :

In [4]:
##### Domain #####

# Domain size
Lx = 2.
Ly = 2.

# Physical grid size
Nx_phys = 40
Ny_phys = 40
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

Initialisation des champs de vitesse / champ de pression et champ de densité :

On note $u,v$ les composantes horizontale, verticales du champ de vitesse. On les initialise à :...

In [5]:
##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))

vrand_ampli = 0.1
#v += vrand_ampli * ( 1-2*np.random.rand(Nx,Ny) ) # random vertical velocity to speed up the instability

# Pressure field press[i,j] and its gradient (dx_press,dy_press)
press = np.zeros((Nx,Ny))
dx_press = np.zeros((Nx,Ny))
dy_press = np.zeros((Nx,Ny))

# Density field
ρ0 = 5
ρ = ρ0*np.ones((Nx,Ny))
#ρ[:, 4*Ny//10 : 8*Ny//10] = 1
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.5, 5, 1)
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.3, 1, ρ[1:-1,1:-1])
ρ[1:-1,1:-1] = np.transpose(np.where(((xx-Lx/2)**2+(yy-0.15*Ly)**2)<0.2**2, 1, 5))

On impose les conditions aux bords (sur les GhostPoints) :

In [6]:
def VelocityGhostPoints(u,v):
    # ya pas cette histoire de "frontière = au milieu des cellules de bord ?"
    ### left
    u[0, :] = u[1, :]
    v[0, :] = v[1, :]
    ### right    
    u[-1, :] = u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = u[:, -2]
    v[:, -1] = v[:, -2]
def VelocityGhostPointsFlatten(u,v):
    ### left
    u[0:Nx],v[0:Nx] = u[Nx:2*Nx],v[Nx:2*Nx]
    ### right
    u[-Nx:],v[-Nx:] = u[-2*Nx:-Nx],v[-2*Nx:-Nx]
    ### bottom               
    u[0::Ny],v[0::Ny] = u[1::Ny],v[0::Ny]
    ### top            
    u[Nx-1::Ny],v[Nx-1::Ny] = u[Nx-2::Ny],v[Nx-1::Ny]
def PhiGhostPoints(phi):
    ### left
    phi[0:Nx] = phi[Nx:2*Nx]
    ### right
    phi[-Nx:] = phi[-2*Nx:-Nx]
    ### bottom               
    phi[0::Ny] = phi[1::Ny]
    ### top            
    phi[Nx-1::Ny] = - phi[Nx-2::Ny]
def get_density_advected(ρ,u,v):
    ρ = ρ.reshape((Nx,Ny))
    # Left:
    ρ_l = np.where(u[1,:]<0,Δt/Δx*(u[1,:]*ρ[1,:]-u[0,:]*ρ[0,:]),Δt/Δx*u[0,:]*(1*-ρ[0,:]))
    ρ[0,:] = ρ_l
    # Right:
    ρ_r = np.where(u[-2,:]>0,Δt/Δx*(u[-2,:]*ρ[-2,:]-u[-1,:]*ρ[-1,:]),Δt/Δx*u[-1,:]*(1*-ρ[-1,:]))
    ρ[-1,:] = ρ_r 
    # Botton:
    ρ_b = np.where(v[:,1]<0,Δt/Δx*(u[:,1]*ρ[:,1]-u[:,0]*ρ[:,0]),Δt/Δx*u[:,0]*(1*-ρ[:,0]))
    ρ[:,  0] = ρ_b
    # Top:
    ρ_t = np.where(v[:,-2]<0,Δt/Δx*(u[:,-2]*ρ[:,-2]-u[:,-1]*ρ[:,-1]),Δt/Δx*u[:,-1]*(1*-ρ[:,-1]))
    ρ[:, -1] = ρ_t
    ρ = ρ.flatten()
def DensityGhostPoints(ρ):
    # ghost points for density : Neumann ∂ρ=0 boundary condition
    ρ[ 0, :] = ρ0 #ρ[1, :] # left
    ρ[-1, :] = ρ0 #ρ[-2, :] # right
    ρ[:,  0] = ρ0 #ρ[:, 1] # bottom
    ρ[:, -1] = ρ0 #ρ[:, -2] # top      

Ensuite vient la définition de notre schéma :

In [7]:
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAP = FD_2D_Laplacian_matrix (Nx, Ny, Δx, Δy, BCdir_left=False, BCdir_right=False, BCdir_top=True, BCdir_bot=False) 


In [8]:
### Méthode de l'article :
## Matrices utiles:
Nx = 42
Ny = 42
datax = [np.ones(Nx), np.ones(Nx), np.ones(Nx)]
datay = [np.ones(Ny), np.ones(Ny), np.ones(Ny)]
offsets = np.array([-1,0,1])
dx,dy = 1/Δx/2, 1/Δy/2
DX = sp.diags(datax,offsets, shape=(Nx,Nx),format = 'csr') * dx
DY = sp.diags(datay,offsets, shape=(Ny,Ny),format = 'csr') * dy
Deltax = sp.kron(sp.eye(Ny,Ny), DX,format = 'csr')
Deltay = sp.kron(DY, sp.eye(Nx,Nx),format = 'csr')
# Retourne la matrice correspondant à (X.Nablda). Ici sous version 2D, prend Nx, Ny la taille de la grille
def grad(s,Δx,Δy,Nx,Ny):
    return Deltax.dot(s),Deltay.dot(s)

def advect(u,v,Δx,Δy,Nx,Ny):
    dNx = 1/Δx/2 * (u*np.eye(Nx,Nx,k=-1) - u*np.eye(Nx,Nx,k=1))
    dNy = 1/Δy/2*(v*np.eye(Ny,Ny,k=-1)-v*np.eye(Ny,Ny,k=1))
    return sp.kron(sp.eye(Ny,Ny), dNx,format = 'csr') + sp.kron(dNy, sp.eye(Nx,Nx),format = 'csr')

def get_rho(u,v,ρ,Nx,Ny,Δx,Δy,Δt):#prend des entrées flattens pour la densité
    A = np.eye(Nx*Ny)+Δt*advect(u,v,Δx,Δy,Nx,Ny)
    return np.linalg.solve(A,ρ)

def approx_u(u,v,ρ,ρ_,s,fx,fy,Δx,Δy,Δt,LAP,Nx,Ny): #prend des entrées flattens pour la densité
    Re = 100000
    A = np.eye(Nx*Ny) + ρ*Δt/ρ_ * advect(u,v,Δx,Δy,Nx,Ny) - Δt/Re * LAP #attention au multiplication par le vecteur colonne...
    Nabsx,Nabsy = grad(s,Δx,Δy,Nx,Ny)
    u_hatx = lg.spsolve(A,Δt*fx-Δt/Re * Nabsx + u.flatten())
    u_haty = lg.spsolve(A,Δt*fy-Δt/Re * Nabsy + v.flatten())
    return u_hatx,u_haty

def get_phi(ρ,u_hatx,u_haty,Δx,Δy,Nx,Ny,LAP):
    ρ_inv = sp.diags([1/ρ],[0], shape = (Nx*Ny,Nx*Ny),format = 'csr')
    LapPart = ρ_inv * LAP
    Dx_ρ, Dy_ρ = grad(1/ρ,Δx,Δy,Nx,Ny)
    Mat = dx * Dx_ρ*Deltax+ dy * Dy_ρ*Deltay + LapPart
    RHS = - Divergence_flat(u_hatx,u_haty, Δx,Δy)
    phi = lg.spsolve(Mat, RHS)
    return phi

def update_u_s(u,v,s,ρ,phi,u_hatx,u_haty,Δx,Δy):
    dX,dY = grad(phi,Δx,Δy,Nx,Ny)
    u = u_hatx + 1/ρ * dX
    v = u_haty + 1/ρ * dY
    s = s - Divergence_flat(u_hatx,u_haty, Δx,Δy)

In [9]:
## Check mass
def mass_check(rho):
    return Δx*Δy*np.sum(rho)

mass_check(ρ)

21.530000000000005

In [10]:
def Divergence_flat (u,v, Δx,Δy):
    div = np.empty((Nx,Ny))
    u_r, v_r = u.reshape((Nx,Ny)), v.reshape((Nx,Ny))
    div[1:-1,1:-1] = ((u_r[2:, 1:-1] - u_r[:-2, 1:-1]) /Δx/2 + (v_r[1:-1, 2:] - v_r[1:-1, :-2]) /Δy/2)
    return div.flatten()

Paramètres de la boucle :

In [11]:
### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.05

t = 0. # total time

# parameters
Re = 10000 # Reynolds number
g = 1
α = 0*pi/4
s = np.zeros((Nx*Ny),dtype = 'float64') #flatten

# how much do we correct for diffusion in semi-lag advection at the price of dispersion
ρ_semilag2_coeff = 0.8
u_semilag2_coeff = 0.7

fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)


niter = 0
disp_modulo = 1

In [12]:
def plot_density(ρ):
    plt.clf()
    ρ_im = ρ.reshape((Nx,Ny))
    plt.imshow(np.transpose(ρ_im[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu, vmin=0, vmax=6)
    plt.show()

In [13]:
# Pour des raisons pratiques, toutes les variables sont constamment flatten.
# On ne reconstruit que pour l'affichage
A = np.ones((Nx*Ny))
fx,fy = +sin(α) * Δt * A, -cos(α) * Δt * A
plt.figure()
while niter<101:
    # Step 1:
    ρ_ = ρ.flatten()
    ρ = get_rho(u,v,ρ_,Nx,Ny,Δx,Δy,Δt)
    get_density_advected(ρ,u,v)
    # Step 2: 
    u_hatx, u_haty = approx_u(u,v,ρ,ρ_,s,fx,fy,Δx,Δy,Δt,LAP,Nx,Ny)
    VelocityGhostPointsFlatten(u_hatx,u_haty)
    # Step 3:
    phi = get_phi(ρ,u_hatx,u_haty,Δx,Δy,Nx,Ny,LAP)
    PhiGhostPoints(phi)
    # Step 4:
    update_u_s(u,v,s,ρ,phi,u_hatx,u_haty,Δx,Δy)
    
    if (niter == 100):
        plot_density(ρ)
        mpl_pause_background(0.00001)
        
    t += Δt
    niter += 1

C:\Users\Paul\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Paul\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\Paul\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\Paul\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Paul\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\Paul\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users

KeyboardInterrupt: 